<a href="https://colab.research.google.com/github/aSafarpoor/KGCLG/blob/main/0_Read_Data_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import random
import numpy as np
from tqdm import tqdm 
import pandas as pd
import json
from collections import Counter
import pandas as pd
import pickle

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/My Drive/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive


In [4]:
# !mkdir KGCLG
%cd KGCLG

/content/drive/My Drive/KGCLG


## read data

In [112]:
!wget https://jmcauley.ucsd.edu/data/amazon_v2/categoryFiles/AMAZON_FASHION.json.gz --no-check-certificate

--2022-10-31 15:54:50--  https://jmcauley.ucsd.edu/data/amazon_v2/categoryFiles/AMAZON_FASHION.json.gz
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 93185167 (89M) [application/x-gzip]
Saving to: ‘AMAZON_FASHION.json.gz’

AMAZON_FASHION.json 100%[===================>]  88.87M  60.3MB/s    in 1.5s    

2022-10-31 15:54:52 (60.3 MB/s) - ‘AMAZON_FASHION.json.gz’ saved [93185167/93185167]



In [117]:
!wget https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_AMAZON_FASHION.json.gz --no-check-certificate

--2022-10-31 15:56:07--  https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_AMAZON_FASHION.json.gz
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 33023658 (31M) [application/x-gzip]
Saving to: ‘meta_AMAZON_FASHION.json.gz’

meta_AMAZON_FASHION 100%[===================>]  31.49M  44.3MB/s    in 0.7s    

2022-10-31 15:56:08 (44.3 MB/s) - ‘meta_AMAZON_FASHION.json.gz’ saved [33023658/33023658]



In [118]:
!gunzip -k meta_AMAZON_FASHION.json.gz
!gunzip -k AMAZON_FASHION.json.gz
!rm meta_AMAZON_FASHION.json.gz
!rm AMAZON_FASHION.json.gz

gzip: AMAZON_FASHION.json.gz: No such file or directory
rm: cannot remove 'AMAZON_FASHION.json.gz': No such file or directory


In [119]:
!ls

AMAZON_FASHION.json	  meta_Software.json
meta_AMAZON_FASHION.json  Software.json


In [154]:
meta = pd.read_json('meta_AMAZON_FASHION.json', lines=True)
ui = pd.read_json('AMAZON_FASHION.json', lines=True)

In [155]:
ui.columns

Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'style', 'image'],
      dtype='object')

In [156]:
print(len(ui))
ui = ui[ui.overall>=4]
ui = ui[['reviewerID','asin']]
print(len(ui))


883636
614807


In [157]:
ui = ui.dropna()
ui = ui.drop_duplicates()
print(len(ui))

608474


In [158]:
ui = ui.reset_index()
del ui['index']

In [159]:
ui.head(5)

,reviewerID,asin
0,A1D4G1SNUZWQOT,7106116521
1,A2MWC41EW7XL15,7106116521
2,A29HLOUW0NS0EH,7106116521
3,A7QS961ROI6E0,7106116521
4,A38NS6NF6WPXS,B00007GDFV


In [160]:
meta.head(3)

,title,brand,feature,rank,date,asin,imageURL,imageURLHighRes,description,price,also_view,also_buy,fit,details,similar_item,tech1
0,Slime Time Fall Fest [With CDROM and Collector...,Group Publishing (CO),[Product Dimensions:\n \n8....,"13,052,976inClothing,Shoesamp;Jewelry(",8.70 inches,0764443682,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XCC Qi promise new spider snake preparing men'...,NaN,NaN,"11,654,581inClothing,Shoesamp;Jewelry(",5 star,1291691480,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Magical Things I Really Do Do Too!,Christopher Manos,[Package Dimensions:\n \n8....,"19,308,073inClothing,ShoesJewelry(",5 star,1940280001,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[For the professional or amateur magician. Ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [161]:
meta.columns

Index(['title', 'brand', 'feature', 'rank', 'date', 'asin', 'imageURL',
       'imageURLHighRes', 'description', 'price', 'also_view', 'also_buy',
       'fit', 'details', 'similar_item', 'tech1'],
      dtype='object')

In [162]:
meta = meta[['asin','description','title','brand',"imageURL","imageURLHighRes"]]

In [163]:
meta = meta.dropna()

In [164]:
meta = meta.reset_index()
del meta['index']

In [165]:
meta.head(5)

,asin,description,title,brand,imageURL,imageURLHighRes
0,1940280001,[For the professional or amateur magician. Ro...,Magical Things I Really Do Do Too!,Christopher Manos,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,9654263246,[When you pull out your extra large carbon fib...,"X. L. Carbon Fiber Money Clip, made in the USA",Roar Carbon,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
2,B00004T3SN,"[A fun addition to any costume party, play, or...",Shimmer Anne Shine Clip On Costume/Halloween C...,Shimmer Anne Shine,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,B00007GDFV,[Authentic crunch leather with rich floral emb...,Buxton Heiress Pik-Me-Up Framed Case,Buxton,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,B0000AWRIF,"[With its smooth surface and high polish, this...",Platinum 4mm Comfort Fit Plain Wedding Band,Amazon Collection,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [166]:
print(len(meta))
meta = meta[meta['description'].map(lambda d: len(d)) > 0]
meta = meta[meta['title'].map(lambda d: len(d)) > 0]
meta = meta[meta['asin'].map(lambda d: len(d)) > 0]
meta = meta[meta['brand'].map(lambda d: len(d)) > 0]
meta = meta[meta['imageURL'].map(lambda d: len(d)) > 0]
meta = meta[meta['imageURLHighRes'].map(lambda d: len(d)) > 0]
print(len(meta))
meta.head(5)

8067
8067


,asin,description,title,brand,imageURL,imageURLHighRes
0,1940280001,[For the professional or amateur magician. Ro...,Magical Things I Really Do Do Too!,Christopher Manos,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,9654263246,[When you pull out your extra large carbon fib...,"X. L. Carbon Fiber Money Clip, made in the USA",Roar Carbon,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
2,B00004T3SN,"[A fun addition to any costume party, play, or...",Shimmer Anne Shine Clip On Costume/Halloween C...,Shimmer Anne Shine,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,B00007GDFV,[Authentic crunch leather with rich floral emb...,Buxton Heiress Pik-Me-Up Framed Case,Buxton,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,B0000AWRIF,"[With its smooth surface and high polish, this...",Platinum 4mm Comfort Fit Plain Wedding Band,Amazon Collection,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


## meta clearance

In [167]:
len(meta),len(ui)

(8067, 608474)

In [168]:
def description_improvement(l):
    s = ' '.join(l)
    return(s)
meta['description'] = meta['description'].apply(description_improvement) 

In [169]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

class Normalizer:
    # based on https://www.geeksforgeeks.org/normalizing-textual-data-with-python/
    def __init__(self):
        # import regex
        self.stop_words = set(stopwords.words('english'))

    def text_normalizer(self,text):
        if type(text) == type([]):
            text = text[0]
        text = str(text)
        # convert to lower case
        lower_string = text.lower()
        # remove numbers
        no_number_string = re.sub(r'\d+','',lower_string)
        # remove all punctuation except words and space
        no_punc_string = re.sub(r'[^\w\s]','', no_number_string)
        # remove white spaces
        no_wspace_string = no_punc_string.strip()
        # convert string to list of words
        lst_string = [no_wspace_string][0].split()
        # remove stopwords
        no_stpwords_string=""
        for i in lst_string:
            if not i in self.stop_words:
                no_stpwords_string += i+' '
        # removing last space
        no_stpwords_string = no_stpwords_string[:-1]
        output = no_stpwords_string
        return output

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [170]:
normalizer = Normalizer()

In [171]:
meta["description"] = meta["description"].apply(normalizer.text_normalizer)
meta["title"] = meta["title"].apply(normalizer.text_normalizer)

In [172]:
def brand_normalizer(s):
    s = s.replace("Brand:","")
    s = s.replace(" ","")
    s = s.split("/")
    return s

In [173]:
meta["brand"] = meta["brand"].apply(brand_normalizer)

In [174]:
def url_normalizer(s):
    return s[0]

In [175]:
meta["imageURL"] = meta["imageURL"].apply(url_normalizer)
meta["imageURLHighRes"] = meta["imageURLHighRes"].apply(url_normalizer)

In [176]:
meta['text'] = meta[['title', 'description','title']].apply(lambda x: ''.join(x), axis=1)

In [177]:
meta.head(5)

,asin,description,title,brand,imageURL,imageURLHighRes,text
0,1940280001,professional amateur magician routines include...,magical things really,[ChristopherManos],https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,magical things reallyprofessional amateur magi...
1,9654263246,pull extra large carbon fiber money clip gives...,x l carbon fiber money clip made usa,[RoarCarbon],https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,x l carbon fiber money clip made usapull extra...
2,B00004T3SN,fun addition costume party play halloween even...,shimmer anne shine clip costumehalloween cat ears,[ShimmerAnneShine],https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,shimmer anne shine clip costumehalloween cat e...
3,B00007GDFV,authentic crunch leather rich floral embossed ...,buxton heiress pikmeup framed case,[Buxton],https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,buxton heiress pikmeup framed caseauthentic cr...
4,B0000AWRIF,smooth surface high polish platinum wedding ba...,platinum mm comfort fit plain wedding band,[AmazonCollection],https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,platinum mm comfort fit plain wedding bandsmoo...


In [178]:
meta = meta.reset_index()
del meta['index']

In [179]:
for i in meta.iloc[1]:
    print(i)

9654263246
pull extra large carbon fiber money clip gives status know whats new people asking get carbon fiber money clip made highest quality carbon fiber america money clip infused epoxy resin giving strong light weight money clip gives ability walk airport detectors put money plastic tray money clip comes year warranty delamination carbon fiber money clips similar sell times buy reason price exceptional number money clips manufactured time
x l carbon fiber money clip made usa
['RoarCarbon']
https://images-na.ssl-images-amazon.com/images/I/41U3SgbuxwL._SR38,50_.jpg
https://images-na.ssl-images-amazon.com/images/I/41U3SgbuxwL.jpg
x l carbon fiber money clip made usapull extra large carbon fiber money clip gives status know whats new people asking get carbon fiber money clip made highest quality carbon fiber america money clip infused epoxy resin giving strong light weight money clip gives ability walk airport detectors put money plastic tray money clip comes year warranty delamination

In [146]:
len(ui)

608474

In [191]:
listmetaasin = list(meta.asin)
def dropextrausers(s):
    if s in listmetaasin:
        return True
    return False

In [196]:
tqdm.pandas()

In [197]:
ui['tf'] = ui['asin'].progress_apply(dropextrausers)

100%|██████████| 608474/608474 [04:40<00:00, 2166.49it/s]


In [199]:
ui = ui[ui['tf']==True]

In [200]:
import pickle


with open('meta.pickle', 'wb') as handle:
    pickle.dump(meta, handle)

with open('ui.pickle', 'wb') as handle:
    pickle.dump(ui, handle)


# with open('filename.pickle', 'rb') as handle:
    # b = pickle.load(handle)